In [1]:
import os, numpy as np
import pandas as pd

os.environ["KERAS_BACKEND"] = "theano"

import keras 
from sklearn import datasets

from sklearn.model_selection import train_test_split

from keras.utils import np_utils


keras.backend.set_floatx('float64')

from keras.utils import np_utils


def set_reproducible():
    import tensorflow as tf
    import random as rn
    import os
    os.environ['PYTHONHASHSEED'] = '1960'
    rn.seed(1960)
    np.random.seed(1960)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K
    tf.set_random_seed(1960)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    # K.set_session(sess)

set_reproducible()


Using Theano backend.


# Build a Keras Model

In [2]:

def create_model():
    from keras.models import Sequential
    from keras.layers.core import Dense, Activation, Dropout
    from keras.utils import np_utils

    model = Sequential()
    model.add(Dense(5, input_shape=(4,) , activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model



In [3]:
iris  = datasets.load_iris()
train_X, test_X, train_y, test_y = train_test_split(iris.data, iris.target, train_size=0.8, test_size=0.2, random_state=1960)


In [4]:

from keras.wrappers.scikit_learn import KerasClassifier

clf = KerasClassifier(build_fn=create_model, epochs=12, verbose=0)

print(train_X.shape , train_y.shape)
clf.fit(train_X, train_y, verbose=0)

(120, 4) (120,)


WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


In [5]:
print(clf.model.__dict__)

{'name': 'sequential_1', 'trainable': True, '_is_compiled': True, '_expects_training_arg': False, '_initial_weights': None, 'supports_masking': False, 'optimizer': <keras.optimizers.Adam object at 0x7fa67c1b8a90>, '_updates': [], '_losses': [], '_per_input_losses': {}, '_per_input_updates': {}, '_layers': [<keras.engine.input_layer.InputLayer object at 0x7fa68d9b0470>, <keras.layers.core.Dense object at 0x7fa6946df588>, <keras.layers.core.Dropout object at 0x7fa67c8a4e80>, <keras.layers.core.Dense object at 0x7fa67c8a4fd0>, <keras.layers.core.Activation object at 0x7fa67c8a4eb8>], '_outbound_nodes': [], '_inbound_nodes': [<keras.engine.base_layer.Node object at 0x7fa67c892ef0>], '_is_graph_network': True, '_uses_inputs_arg': True, 'outputs': [Softmax.0], 'inputs': [/dense_1_input], '_built': True, '_compute_previous_mask': True, '_input_layers': [<keras.engine.input_layer.InputLayer object at 0x7fa68d9b0470>], '_output_layers': [<keras.layers.core.Activation object at 0x7fa67c8a4eb8>],

In [6]:
print(test_X.shape)
preds = clf.predict(test_X[0,:].reshape(1,4))
print(preds)


(30, 4)
[2]


# Generate SQL Code from the Model

In [7]:
import json, requests, base64, dill as pickle, sys



sys.setrecursionlimit(200000)
pickle.settings['recurse'] = False

# no luck for the web service... pickling feature of tensorflow and/or keras objects seems not to be a priority.
# there is a lot of github issues in the two projects when I search for pickle keyword!!!.

def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



def test_sql_gen(keras_regressor , metadata):
    import sklearn2sql.PyCodeGenerator as codegen
    cg1 = codegen.cAbstractCodeGenerator();
    cg1.mOptions.mActivateMaterialization = False
    lSQL = cg1.generateCodeWithMetadata(clf, metadata, dsn = None, dialect = "postgresql");
    return lSQL[1]


In [8]:
# commented .. see above
# pickle_data = pickle.dumps(clf)
# lSQL = test_ws_sql_gen(pickle_data)
# print(lSQL[0:2000])

In [9]:
lMetaData = {}
lMetaData['features'] = list(iris.feature_names);
#column name adaptation
lMetaData['features'] = [col.replace(" (cm)" , "_cm") for col in lMetaData['features'] ]

lMetaData["targets"] = ['TGT']
lMetaData['primary_key'] = 'KEY'
lMetaData['table'] = 'iris'

    
lSQL = test_sql_gen(clf , lMetaData)


cGenerationWrapperFactory::createWrapper() <class 'keras.wrappers.scikit_learn.KerasClassifier'>
cClassifierMixin_CodeGenWrapper::setObject <class 'keras.wrappers.scikit_learn.KerasClassifier'>
BACKEND_DIALECT postgresql
CREATING_DATABASE_BACKEND_DSN_DIALECT 1.2.7 None postgresql
KERAS_GENERATE_EXPRESSION_START <class 'sqlalchemy.sql.selectable.Alias'> <class 'sklearn2sql.GenericModels.Keras.cSerializedKeras'>
{'mKerasData': <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa68d9be588>, 'mFeatureNames': ['sepal length_cm', 'sepal width_cm', 'petal length_cm', 'petal width_cm'], 'mClasses': array([0, 1, 2]), 'mMode': 'Classification'}
{'build_fn': <function create_model at 0x7fa68d9ae048>, 'sk_params': {'epochs': 12, 'verbose': 0}, 'classes_': array([0, 1, 2]), 'n_classes_': 3, 'model': <keras.engine.sequential.Sequential object at 0x7fa6946df518>}
ABSTRACT_GENERATE_CTE keras_input keras_input 5 False False
ABSTRACT_GENERATE_SIMPLE_CTE keras_input 5
GENERATING_LAYER 0 dense_1 <

DUMP_RESULT_KEY_EXPRESSIONS : ['KEY']
DUMP_RESULT_CLASS_EXPRESSIONS : Score 0 arg_max_cte."Score_0"
DUMP_RESULT_CLASS_EXPRESSIONS : Score 1 arg_max_cte."Score_1"
DUMP_RESULT_CLASS_EXPRESSIONS : Score 2 arg_max_cte."Score_2"
DUMP_RESULT_CLASS_EXPRESSIONS : Proba 0 arg_max_cte."Proba_0"
DUMP_RESULT_CLASS_EXPRESSIONS : Proba 1 arg_max_cte."Proba_1"
DUMP_RESULT_CLASS_EXPRESSIONS : Proba 2 arg_max_cte."Proba_2"
DUMP_RESULT_CLASS_EXPRESSIONS : LogProba 0 CASE WHEN (arg_max_cte."Proba_0" IS NULL OR arg_max_cte."Proba_0" > :param_1) THEN ln(arg_max_cte."Proba_0") ELSE :param_2 END
DUMP_RESULT_CLASS_EXPRESSIONS : LogProba 1 CASE WHEN (arg_max_cte."Proba_1" IS NULL OR arg_max_cte."Proba_1" > :param_1) THEN ln(arg_max_cte."Proba_1") ELSE :param_2 END
DUMP_RESULT_CLASS_EXPRESSIONS : LogProba 2 CASE WHEN (arg_max_cte."Proba_2" IS NULL OR arg_max_cte."Proba_2" > :param_1) THEN ln(arg_max_cte."Proba_2") ELSE :param_2 END
DUMP_RESULT_AGGREGATED_CLASS_EXPRESSIONS : best_class_by_Proba arg_max_cte."arg_

In [10]:
print(lSQL)

WITH keras_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."sepal length_cm" AS "sepal length_cm", "ADS"."sepal width_cm" AS "sepal width_cm", "ADS"."petal length_cm" AS "petal length_cm", "ADS"."petal width_cm" AS "petal width_cm" 
FROM iris AS "ADS"), 
keras_input_1 AS 
(SELECT keras_input."KEY" AS "KEY", keras_input."sepal length_cm" AS "sepal length_cm", keras_input."sepal width_cm" AS "sepal width_cm", keras_input."petal length_cm" AS "petal length_cm", keras_input."petal width_cm" AS "petal width_cm" 
FROM keras_input), 
layer_dense_1 AS 
(SELECT keras_input_1."KEY" AS "KEY", -0.038786080459869116 + 0.5680870563197641 * keras_input_1."sepal length_cm" + -0.24175808521441206 * keras_input_1."sepal width_cm" + -0.769862648138741 * keras_input_1."petal length_cm" + -0.41264662502833943 * keras_input_1."petal width_cm" AS output_1, -0.033320941114665034 + -0.3481091447354203 * keras_input_1."sepal length_cm" + 0.5931118430583008 * keras_input_1."sepal width_cm" + -0.3860692210990508 * 

# Execute the SQL Code

In [11]:
# save the dataset in a database table


import sqlalchemy as sa

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(iris.data);
lTable.columns = lMetaData['features']
lTable['TGT'] = iris.target
lTable['KEY'] = range(iris.data.shape[0])
lTable.to_sql(lMetaData['table'] , conn,   if_exists='replace', index=False)

In [12]:
sql_output = pd.read_sql(lSQL , conn);
conn.close()

In [13]:
sql_output.sample(12, random_state=1960)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,None,None,None,0.003947,0.038433,0.957620,-5.534848,-3.258830,-0.043304,2,0.957620
74,74,None,None,None,0.031859,0.111739,0.856402,-3.446438,-2.191592,-0.155015,2,0.856402
9,9,None,None,None,0.217764,0.390962,0.391274,-1.524344,-0.939144,-0.938348,2,0.391274
88,88,None,None,None,0.047042,0.137910,0.815048,-3.056710,-1.981153,-0.204509,2,0.815048
25,25,None,None,None,0.202902,0.376636,0.420462,-1.595034,-0.976476,-0.866400,2,0.420462
5,5,None,None,None,0.209543,0.362327,0.428130,-1.562825,-1.015208,-0.848329,2,0.428130
48,48,None,None,None,0.222019,0.398747,0.379234,-1.504991,-0.919428,-0.969603,1,0.398747
117,117,None,None,None,0.002039,0.024917,0.973044,-6.195408,-3.692208,-0.027326,2,0.973044
83,83,None,None,None,0.007435,0.053977,0.938588,-4.901595,-2.919196,-0.063378,2,0.938588
105,105,None,None,None,0.000925,0.016776,0.982299,-6.985626,-4.087782,-0.017860,2,0.982299


# Keras Prediction

In [14]:
keras_output = pd.DataFrame()
keras_output_key = pd.DataFrame(list(range(iris.data.shape[0])), columns=['KEY']);
keras_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
keras_output_proba = pd.DataFrame(clf.predict_proba(iris.data), columns=['Proba_0', 'Proba_1', 'Proba_2'])
keras_output = pd.concat([keras_output_key, keras_output_score, keras_output_proba] , axis=1)
for class_label in [0, 1, 2]:
    keras_output['LogProba_' + str(class_label)] = np.log(keras_output_proba['Proba_' + str(class_label)])
keras_output['Decision'] = clf.predict(iris.data)
keras_output.sample(12, random_state=1960)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.003947,0.038433,0.957620,-5.534848,-3.258830,-0.043304,2
74,74,NaN,NaN,NaN,0.031859,0.111739,0.856402,-3.446438,-2.191592,-0.155015,2
9,9,NaN,NaN,NaN,0.217764,0.390962,0.391274,-1.524344,-0.939144,-0.938348,2
88,88,NaN,NaN,NaN,0.047042,0.137910,0.815048,-3.056710,-1.981153,-0.204509,2
25,25,NaN,NaN,NaN,0.202902,0.376636,0.420462,-1.595034,-0.976476,-0.866400,2
5,5,NaN,NaN,NaN,0.209543,0.362327,0.428130,-1.562825,-1.015208,-0.848329,2
48,48,NaN,NaN,NaN,0.222019,0.398747,0.379234,-1.504991,-0.919428,-0.969603,1
117,117,NaN,NaN,NaN,0.002039,0.024917,0.973044,-6.195408,-3.692208,-0.027326,2
83,83,NaN,NaN,NaN,0.007435,0.053977,0.938588,-4.901595,-2.919196,-0.063378,2
105,105,NaN,NaN,NaN,0.000925,0.016776,0.982299,-6.985626,-4.087782,-0.017860,2


# Comparing the SQL and Keras Predictions

In [15]:
sql_keras_join = keras_output.join(sql_output , how='left', on='KEY', lsuffix='_keras', rsuffix='_sql')


In [16]:
sql_keras_join.head(12)

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Proba_0_keras,Proba_1_keras,Proba_2_keras,LogProba_0_keras,LogProba_1_keras,LogProba_2_keras,...,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql,DecisionProba
0,0,NaN,NaN,NaN,0.225168,0.402078,0.372754,-1.490908,-0.911110,-0.986836,...,None,None,0.225168,0.402078,0.372754,-1.490908,-0.911110,-0.986836,1,0.402078
1,1,NaN,NaN,NaN,0.213669,0.396110,0.390221,-1.543329,-0.926062,-0.941042,...,None,None,0.213669,0.396110,0.390221,-1.543329,-0.926062,-0.941042,1,0.396110
2,2,NaN,NaN,NaN,0.231637,0.395732,0.372631,-1.462584,-0.927017,-0.987167,...,None,None,0.231637,0.395732,0.372631,-1.462584,-0.927017,-0.987167,1,0.395732
3,3,NaN,NaN,NaN,0.223400,0.367733,0.408867,-1.498790,-1.000399,-0.894365,...,None,None,0.223400,0.367733,0.408867,-1.498790,-1.000399,-0.894365,2,0.408867
4,4,NaN,NaN,NaN,0.232461,0.395762,0.371777,-1.459031,-0.926943,-0.989462,...,None,None,0.232461,0.395762,0.371777,-1.459031,-0.926943,-0.989462,1,0.395762
5,5,NaN,NaN,NaN,0.209543,0.362327,0.428130,-1.562825,-1.015208,-0.848329,...,None,None,0.209543,0.362327,0.428130,-1.562825,-1.015208,-0.848329,2,0.428130
6,6,NaN,NaN,NaN,0.234303,0.369576,0.396121,-1.451139,-0.995399,-0.926036,...,None,None,0.234303,0.369576,0.396121,-1.451139,-0.995399,-0.926036,2,0.396121
7,7,NaN,NaN,NaN,0.221152,0.385749,0.393099,-1.508907,-0.952568,-0.933694,...,None,None,0.221152,0.385749,0.393099,-1.508907,-0.952568,-0.933694,2,0.393099
8,8,NaN,NaN,NaN,0.226738,0.371081,0.402181,-1.483960,-0.991335,-0.910853,...,None,None,0.226738,0.371081,0.402181,-1.483960,-0.991335,-0.910853,2,0.402181
9,9,NaN,NaN,NaN,0.217764,0.390962,0.391274,-1.524344,-0.939144,-0.938348,...,None,None,0.217764,0.390962,0.391274,-1.524344,-0.939144,-0.938348,2,0.391274


In [17]:
condition = (sql_keras_join.Decision_sql != sql_keras_join.Decision_keras)
sql_keras_join[condition]

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Proba_0_keras,Proba_1_keras,Proba_2_keras,LogProba_0_keras,LogProba_1_keras,LogProba_2_keras,...,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql,DecisionProba
